In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from tqdm.notebook import tqdm
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler, TensorDataset, random_split, SubsetRandomSampler, ConcatDataset

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold

# Load Data

In [3]:
transform = transforms.Compose([transforms.Resize((227, 227)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

batch_size = 64

train_set = torchvision.datasets.ImageFolder('../input/melanoma-skin-cancer-dataset-of-10000-images/melanoma_cancer_dataset/train', 
                                            transform=transform)

test_set = torchvision.datasets.ImageFolder('../input/melanoma-skin-cancer-dataset-of-10000-images/melanoma_cancer_dataset/test', 
                                            transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, 
                                          shuffle=True, num_workers=2)

classes = train_set.classes

classes

['benign', 'malignant']

# Instantiate CNN
### Replicating AlexNet for its performance
* Info: https://en.wikipedia.org/wiki/AlexNet
* Disclaimers: 
    * AlexNet has many layers and is computationally expensive so use cuda to speed up training
    * Can also try pretrained models, but this is for educational purposes

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4, padding=2),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(kernel_size=3, stride=2),
                                      nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, padding=2),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(kernel_size=3, stride=2),
                                      nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(kernel_size=3, stride=2),
                                      nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(in_channels=512, out_channels=256, kernel_size=3, padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(kernel_size=3, stride=2),
                                      nn.AdaptiveAvgPool2d((6, 6))
                                     )
        
        self.classifier = nn.Sequential(nn.Linear(256 * 6 * 6, 4096),
                                        nn.Dropout(p=0.5),
                                        nn.Linear(4096, 4096),
                                        nn.Dropout(p=0.5),
                                        nn.Linear(4096, 2)
                                       )


    def forward(self, x):
        x = self.features(x)
        
        x = x.view(-1, 6 * 6 * 256)
        
        x = self.classifier(x)
        return x


model = AlexNet()
model.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): AdaptiveAvgPool2d(output_size=(6, 6))
  )
  (classifier): Sequential(
    (0): Linear(in_fea

# Define Loss Function
* CrossEntropyLoss() includes softmax (doesn't need to be defined in CNN)

In [6]:
criterion = nn.CrossEntropyLoss()

# K-Fold Cross Validation
* Split train set into train and validation sets for training before predicting on test set to catch overfitting

In [7]:
torch.manual_seed(42)

#dataset = ConcatDataset([train_set, test_set])
dataset = train_set

num_epochs = 30

k = 10

splits = KFold(n_splits = k, shuffle = True, random_state = 42)

foldperf = {}

In [8]:
def train_epoch(model, device, dataloader, loss_fn, optimizer):
    train_loss, train_correct = 0.0, 0
    model.train()
    for images, labels in dataloader:

        images,labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = loss_fn(output,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        scores, predictions = torch.max(output.data, 1)
        train_correct += (predictions == labels).sum().item()

    return train_loss, train_correct
  
def valid_epoch(model, device, dataloader, loss_fn):
    valid_loss, val_correct = 0.0, 0
    model.eval()
    for images, labels in dataloader:

        images, labels = images.to(device), labels.to(device)
        output = model(images)
        loss = loss_fn(output,labels)
        valid_loss += loss.item() * images.size(0)
        scores, predictions = torch.max(output.data,1)
        val_correct += (predictions == labels).sum().item()

    return valid_loss, val_correct

# Train Model

In [9]:
for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):

    print('Fold {}'.format(fold + 1))

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size = batch_size, sampler = train_sampler)
    test_loader = DataLoader(dataset, batch_size = batch_size, sampler = test_sampler)
    
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.001)

    history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

    for epoch in tqdm(range(num_epochs)):
        train_loss, train_correct=train_epoch(model, device, train_loader, criterion, optimizer)
        test_loss, test_correct=valid_epoch(model, device, test_loader, criterion)

        train_loss = train_loss / len(train_loader.sampler)
        train_acc = train_correct / len(train_loader.sampler) * 100
        test_loss = test_loss / len(test_loader.sampler)
        test_acc = test_correct / len(test_loader.sampler) * 100

        print("Epoch:{}/{} || AVG Training Loss:{:.3f} || AVG Test Loss:{:.3f} || AVG Training Acc {:.2f} % || AVG Test Acc {:.2f} %".format(epoch + 1,
                                                                                                             num_epochs,
                                                                                                             train_loss,
                                                                                                             test_loss,
                                                                                                             train_acc,
                                                                                                             test_acc))
        history['train_loss'].append(train_loss)
        history['test_loss'].append(test_loss)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)

    foldperf['fold{}'.format(fold+1)] = history
    
torch.save(model,'melanoma_CNN.pt') 

Fold 1


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch:1/30 || AVG Training Loss:0.678 || AVG Test Loss:0.608 || AVG Training Acc 60.38 % || AVG Test Acc 72.63 %
Epoch:2/30 || AVG Training Loss:0.431 || AVG Test Loss:0.485 || AVG Training Acc 78.70 % || AVG Test Acc 78.67 %
Epoch:3/30 || AVG Training Loss:0.349 || AVG Test Loss:0.457 || AVG Training Acc 84.41 % || AVG Test Acc 80.96 %
Epoch:4/30 || AVG Training Loss:0.308 || AVG Test Loss:0.307 || AVG Training Acc 87.04 % || AVG Test Acc 86.06 %
Epoch:5/30 || AVG Training Loss:0.286 || AVG Test Loss:0.299 || AVG Training Acc 87.91 % || AVG Test Acc 86.89 %
Epoch:6/30 || AVG Training Loss:0.278 || AVG Test Loss:0.280 || AVG Training Acc 88.52 % || AVG Test Acc 87.83 %
Epoch:7/30 || AVG Training Loss:0.266 || AVG Test Loss:0.307 || AVG Training Acc 89.14 % || AVG Test Acc 86.99 %
Epoch:8/30 || AVG Training Loss:0.266 || AVG Test Loss:0.306 || AVG Training Acc 89.15 % || AVG Test Acc 86.99 %
Epoch:9/30 || AVG Training Loss:0.259 || AVG Test Loss:0.273 || AVG Training Acc 89.68 % || AVG 

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch:1/30 || AVG Training Loss:0.151 || AVG Test Loss:0.168 || AVG Training Acc 94.09 % || AVG Test Acc 93.03 %
Epoch:2/30 || AVG Training Loss:0.148 || AVG Test Loss:0.160 || AVG Training Acc 94.13 % || AVG Test Acc 93.76 %
Epoch:3/30 || AVG Training Loss:0.135 || AVG Test Loss:0.146 || AVG Training Acc 94.75 % || AVG Test Acc 94.38 %
Epoch:4/30 || AVG Training Loss:0.134 || AVG Test Loss:0.164 || AVG Training Acc 94.75 % || AVG Test Acc 93.44 %
Epoch:5/30 || AVG Training Loss:0.126 || AVG Test Loss:0.158 || AVG Training Acc 94.82 % || AVG Test Acc 93.34 %
Epoch:6/30 || AVG Training Loss:0.130 || AVG Test Loss:0.154 || AVG Training Acc 94.93 % || AVG Test Acc 94.28 %
Epoch:7/30 || AVG Training Loss:0.109 || AVG Test Loss:0.140 || AVG Training Acc 95.87 % || AVG Test Acc 95.01 %
Epoch:8/30 || AVG Training Loss:0.114 || AVG Test Loss:0.154 || AVG Training Acc 95.38 % || AVG Test Acc 94.17 %
Epoch:9/30 || AVG Training Loss:0.106 || AVG Test Loss:0.211 || AVG Training Acc 96.00 % || AVG 

...

Epoch:1/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:2/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:3/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:4/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:5/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:6/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:7/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:8/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:9/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch:1/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:2/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:3/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:4/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:5/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:6/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:7/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:8/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:9/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch:1/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:2/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:3/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:4/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:5/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:6/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:7/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:8/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc 100.00 % || AVG Test Acc 100.00 %
Epoch:9/30 || AVG Training Loss:0.000 || AVG Test Loss:0.000 || AVG Training Acc

# Predict Test Set

In [10]:
nb_classes = 2

test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, 
                                          shuffle=True, num_workers=2)

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    test_running_corrects = 0.0
    test_total = 0.0
    model.eval()
    for i, (test_inputs, test_labels) in enumerate(test_loader, 0):
        test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()

        test_outputs = model(test_inputs)
        _, test_outputs = torch.max(test_outputs, 1)
        
        test_total += test_labels.size(0)
        test_running_corrects += (test_outputs == test_labels).sum().item()
        
        for t, p in zip(test_labels.view(-1), test_outputs.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1
        
        
    print(f'Testing Accuracy: {(100 * test_running_corrects / test_total)}%')
print(f'Confusion Matrix:\n {confusion_matrix}')

Testing Accuracy: 91.9%
Confusion Matrix:
 tensor([[472.,  28.],
        [ 53., 447.]])


# F1 Score
 * Model accuracy only tells you how many correct or incorrect predictions the model made, but this isn't always the best metric
    * Using F1 score will factor your model's precision and recall
        * precision: detects type 1 errors or false positives i.e. (model predicts malignant when true value is benign)
        * recall: detects type 2 errors or false negatives i.e. (model predicts benign when true value is malignant)

In [11]:
precision = confusion_matrix[0][0] / (confusion_matrix[0][0] + confusion_matrix[1][0])
recall = confusion_matrix[0][0] / (confusion_matrix[0][0] + confusion_matrix[0][1])

f1_score = 2 * precision * recall / (precision + recall)
print(f'F1 Score: {f1_score: .3f}')

F1 Score:  0.921
